In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import os
from getpass import getpass
from textwrap import dedent
from pprint import pprint

from langchain_openai import ChatOpenAI

from core.state import ReWOO
from core.planner import planner_agent, Planner
from core.scheduler import Executor
from core.worker import worker_agent, Worker
from core.solver import solver_agent, Solver

from langgraph.graph import StateGraph, END

In [108]:
os.environ["OPENAI_API_KEY"] = getpass("OPENAI API Key:")

In [109]:
query = "What is the hometown of the 2024 australian open winner?"

In [110]:
gpt35turbo_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

In [111]:
planner = Planner(
    llm=gpt35turbo_llm,
    few_shot=dedent("""
    For example:

    Task: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
    Plan: Search for more information about Colorado orogeny.
    #E1 = GoogleSearch[Colorado orogeny]
    Plan: Find out the area that eastern sector of the Colorado orogeny extends into.
    #E2 = LLM[What is the name of the area that eastern sector of Colorado extends into? Given context: #E1]
    Plan: Search for more information about the area.
    #E3 = GoogleSearch[#E2]
    Plan: Find out the elevation range for the area.
    #E4 = LLM[What is elevation range for the area #E2? Given context: #E3]
    
    Task: What profession does Nicholas Ray and Elia Kazan have in common?
    Plan: Search for more information about Nicholas Ray.
    #E1 = GoogleSearch[Nicholas Ray]
    Plan: Search for more information about Elia Kazan.
    #E2 = GoogleSearch[Elia Kazan]
    Plan: Compare the two and determine what profession they have in common.
    #E3 = LLM[What profession does Nicholas Ray and Elia Kazan have in common? Given context: #E1, #E2]
    """)
)

In [177]:
from typing import Any
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.language_models.chat_models import BaseChatModel

os.environ["TAVILY_API_KEY"] = getpass("TAVILY API Key:")

class CustomSearchTool(TavilySearchResults):
    name = "GoogleSearch"
    description = "Worker that searches results from Google. \
        Useful when you need to find short and succinct answers about a specific topic. \
        The input should be a search query."


class CustomLLMTool(BaseTool):
    name = "LLM"
    description = "A pretrained LLM like yourself. \
        Useful when you need to act with general world knowledge and common sense. \
        Prioritize it when you are confident in solving the problem yourself. \
        Input can be any instruction."

    llm: BaseChatModel = None

    def __init__(self, *args, llm, **kwargs):
        super().__init__(*args, **kwargs)
        self.llm = llm
    
    def _run(self, input):
        return self.llm.invoke(input).content
        
tools = [CustomSearchTool(), CustomLLMTool(llm=gpt35turbo_llm)]

In [178]:
worker = Worker(
    scheduler=Executor()
)

In [179]:
solver = Solver(
    llm=gpt35turbo_llm
)

In [180]:
flow = StateGraph(ReWOO)
flow.add_node("planner_node", planner_agent)
flow.add_node("worker_node", worker_agent)
flow.add_edge("planner_node", "worker_node")
flow.add_node("solver_node", solver_agent)
flow.add_edge("worker_node", "solver_node")
flow.add_edge("solver_node", END)
flow.set_entry_point("planner_node")
app = flow.compile()

In [181]:
# Iterate over the stream of outputs
for output in app.stream(ReWOO(query=query, tools=tools, planner=planner, worker=worker, solver=solver)):
    # Each output is a dictionary where keys are node names and values are outputs
    for node, state_output in output.items():
        pprint(node)
        pprint(state_output)

'planner_node'
{'generation': None,
 'planner': <core.planner.Planner object at 0x7fe623e66be0>,
 'query': 'What is the hometown of the 2024 australian open winner?',
 'solver': <core.solver.Solver object at 0x7fe623689ac0>,
 'tools': [CustomSearchTool(),
           CustomLLMTool(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe624077910>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe623e59fd0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''))],
 'worker': <core.worker.Worker object at 0x7fe6240a9850>}
'worker_node'
{'generation': None,
 'planner': <core.planner.Planner object at 0x7fe623e66be0>,
 'query': 'What is the hometown of the 2024 australian open winner?',
 'solver': <core.solver.Solver object at 0x7fe623689ac0>,
 'tools': [CustomSearchTool(),
           CustomLLMTool(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe624077910>, async_client=<openai

In [174]:
pprint(state_output["planner"].plans)

{'#E1': {'args': '2024 Australian Open winner',
         'description': 'Search for information about the 2024 Australian Open '
                        'winner.',
         'evidence': "[{'url': "
                     "'https://www.cbssports.com/tennis/news/australian-open-2024-jannik-sinner-aryna-sabalenka-crowned-as-grand-slam-singles-champions-at-melbourne-park/', "
                     '\'content\': "Qinwen Zheng, 6-3, 6-2\\nOur Latest Tennis '
                     'Stories\\nSinner, Sabalenka win Australian Open singles '
                     'titles\\nSinner makes epic comeback to win Australian '
                     'Open\\n2024 Australian Open odds, Sinner vs. Medvedev '
                     'picks\\nSabalenka defeats Zheng to win 2024 Australian '
                     'Open\\n2024 Australian Open odds, Sabalenka vs. Zheng '
                     'picks\\n2024 Australian Open odds, Medvedev vs. Zverev '
                     'picks\\nAustralian Open odds: Djokovic vs. Sinner pic

In [182]:
pprint(state_output["generation"])

{'context': 'Plan: Search for information about the 2024 Australian Open '
            'winner.\n'
            "Evidence: [{'url': "
            "'https://www.cbssports.com/tennis/news/australian-open-2024-jannik-sinner-claims-first-grand-slam-title-in-epic-comeback-win-over-daniil-medvedev/', "
            '\'content\': \'"\\nOur Latest Tennis Stories\\nSinner makes epic '
            'comeback to win Australian Open\\nSinner, Sabalenka win '
            'Australian Open singles titles\\n2024 Australian Open odds, '
            'Sinner vs. Medvedev picks\\nSabalenka defeats Zheng to win 2024 '
            'Australian Open\\n2024 Australian Open odds, Sabalenka vs. Zheng '
            'picks\\n2024 Australian Open odds, Medvedev vs. Zverev '
            'picks\\nAustralian Open odds: Djokovic vs. Sinner picks, '
            'bets\\nAustralian Open odds: Gauff vs. Sabalenka picks, '
            'bets\\nAustralian Open odds: Zheng vs. Yastremska picks, '
            "bets\\nNick Kyrgios 